# Accuracy baselines

Computation of the insertion and deletion scores on a testbench of explanations for 100 images of the ImageNet training dataset.

In [1]:
!pip install xplique
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import pandas as pd
import os
from google.colab import drive
import json
import itertools

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from xplique.attributions import SobolAttributionMethod, Rise # black box methods
from xplique.attributions import Saliency, GradientInput, IntegratedGradients, GradCAMPP, VarGrad# white box methods
from xplique.metrics import Deletion, Insertion, MuFidelity # evaluation methods
from spectral_sobol.tf_explainer import WaveletSobol
import keras.applications as app # models

In [3]:
# load the labels and the images
source_dir = '/content/gdrive/MyDrive/sam-benchmark/imagenet'
target_dir = '/content/gdrive/MyDrive/sam-benchmark'

# number of iterations to compute the scores
reps = 1
nb_design = 4
grid_size = 28
batch_size = 128

# labels
labels = pd.read_csv(os.path.join(source_dir, 'labels.csv'))
y_wavelet = labels['label'].values
y = np.array([tf.keras.utils.to_categorical(label, 1000) for label in y_wavelet])

# images
images = [cv2.imread(os.path.join(source_dir, img_name))[..., ::-1] for img_name in labels['name'].values]


# helper that will update the file if it exists already
def update(target_dir, filename, data):
  """
  updates the file if it exists, creates and save otherwise
  """

  exists = os.path.exists(os.path.join(target_dir, filename))

  if exists:
    old = json.load(open(os.path.join(target_dir, filename)))
    
    # add the items
    for item in data.keys():
      old[item] = data[item]

    # save the file
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(old, f)

  else:
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(data, f)

# model set up
# each value contains the model and its preprocessing module
models = {
    'VGG'          : (app.VGG16(classifier_activation = 'softmax'), app.vgg16.preprocess_input),
    'ResNet'       : (app.ResNet50(classifier_activation = 'softmax'), app.resnet.preprocess_input),
    'MobileNet'    : (app.MobileNet(classifier_activation = 'softmax'), app.mobilenet.preprocess_input),
    'EfficientNet' : (app.EfficientNetV2B0(classifier_activation = 'softmax'), app.efficientnet.preprocess_input)
}

29403144/29403144 [==============================] - 2s 0us/step


In [4]:
# computation

scores = {}
batch_size = 256
for name in models.keys():

  print('Starting model {}'.format(name))

  model, preprocessing = models[name]
  model.layers[-1].activation = tf.keras.activations.softmax # 

  # preprocess the data
  x = preprocessing(np.array(images, copy=True))

  # initialize the metrics
  # insertion = Insertion(model, x, y)
  # deletion = Deletion(model, x, y)
  mufidelity = MuFidelity(model, x, y, batch_size, nb_samples = 25)

  # set up the explanations
  explainers = {
      'saliency'   : Saliency(model),
      'grad-inp'   : GradientInput(model),
      'grad-integ' : IntegratedGradients(model, steps=80),
      'GradCAMPP'  : GradCAMPP(model),
      'VarGrad'    : VarGrad(model, nb_samples=80),
      'rise'  : Rise(model, nb_samples=4000),
      'sobol' : SobolAttributionMethod(model)
  }

  print('Explanations set up complete.')

  scores[name] = {}

  # compute the different explanations for this backbone
  for explanation in explainers.keys():

    explainer = explainers[explanation]
    out = explainer(x,y)
    
    scores[name][explanation] = {
        #'insertion' : insertion(out),
        #'deletion'  : deletion(out)
        'mu_fidelity' : mufidelity(out)
    }

    print(scores)

    update(target_dir, 'benchmark_mu_fidelity.json', scores)
    print('Computation and evaluation of the explanation {} complete.'.format(explanation))


  print('Model {} complete'.format(name))



Starting model VGG
Explanations set up complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}}}
Computation and evaluation of the explanation saliency complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}}}
Computation and evaluation of the explanation grad-inp complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}}}
Computation and evaluation of the explanation grad-integ complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}}}
Computation and evaluation of the explanation GradCAMPP complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367

{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}, 'VarGrad': {'mu_fidelity': 0.053984869895904286}, 'rise': {'mu_fidelity': 0.02046168495350125}, 'sobol': {'mu_fidelity': 0.0946843889879456}}, 'ResNet': {'saliency': {'mu_fidelity': 0.059841692817237}, 'grad-inp': {'mu_fidelity': 0.050655305880740194}, 'grad-integ': {'mu_fidelity': 0.11188217096801097}}}
Computation and evaluation of the explanation grad-integ complete.


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}, 'VarGrad': {'mu_fidelity': 0.053984869895904286}, 'rise': {'mu_fidelity': 0.02046168495350125}, 'sobol': {'mu_fidelity': 0.0946843889879456}}, 'ResNet': {'saliency': {'mu_fidelity': 0.059841692817237}, 'grad-inp': {'mu_fidelity': 0.050655305880740194}, 'grad-integ': {'mu_fidelity': 0.11188217096801097}, 'GradCAMPP': {'mu_fidelity': 0.08346212106632754}}}
Computation and evaluation of the explanation GradCAMPP complete.
{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}, 'VarGrad': {'mu_fidelity': 0.053984869895904286}, 'rise': {'mu_fidelity': 0.02046168495350125}, 'sobol': {'mu_fidelity': 0.0946843889879456}}, 'ResNet': {'

{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}, 'VarGrad': {'mu_fidelity': 0.053984869895904286}, 'rise': {'mu_fidelity': 0.02046168495350125}, 'sobol': {'mu_fidelity': 0.0946843889879456}}, 'ResNet': {'saliency': {'mu_fidelity': 0.059841692817237}, 'grad-inp': {'mu_fidelity': 0.050655305880740194}, 'grad-integ': {'mu_fidelity': 0.11188217096801097}, 'GradCAMPP': {'mu_fidelity': 0.08346212106632754}, 'VarGrad': {'mu_fidelity': 0.09911691753667765}, 'rise': {'mu_fidelity': 0.07413847662213993}, 'sobol': {'mu_fidelity': 0.10770710104906747}}, 'MobileNet': {'saliency': {'mu_fidelity': -0.002230769230769222}, 'grad-inp': {'mu_fidelity': 0.023269230769230768}, 'grad-integ': {'mu_fidelity': 0.12965384615384618}}}
Computation and evaluation of the explanation grad-integ complete.


{'VGG': {'saliency': {'mu_fidelity': 0.04301891189026125}, 'grad-inp': {'mu_fidelity': 0.10508075813289325}, 'grad-integ': {'mu_fidelity': 0.1367776318612683}, 'GradCAMPP': {'mu_fidelity': 0.08914598959758953}, 'VarGrad': {'mu_fidelity': 0.053984869895904286}, 'rise': {'mu_fidelity': 0.02046168495350125}, 'sobol': {'mu_fidelity': 0.0946843889879456}}, 'ResNet': {'saliency': {'mu_fidelity': 0.059841692817237}, 'grad-inp': {'mu_fidelity': 0.050655305880740194}, 'grad-integ': {'mu_fidelity': 0.11188217096801097}, 'GradCAMPP': {'mu_fidelity': 0.08346212106632754}, 'VarGrad': {'mu_fidelity': 0.09911691753667765}, 'rise': {'mu_fidelity': 0.07413847662213993}, 'sobol': {'mu_fidelity': 0.10770710104906747}}, 'MobileNet': {'saliency': {'mu_fidelity': -0.002230769230769222}, 'grad-inp': {'mu_fidelity': 0.023269230769230768}, 'grad-integ': {'mu_fidelity': 0.12965384615384618}, 'GradCAMPP': {'mu_fidelity': -0.0006347212784962081}}}
Computation and evaluation of the explanation GradCAMPP complete.
